In [ ]:
def time_to_int(time):
  if (time == float):
    return time
  time_str = str(time)
  time_series = time_str.rsplit(':')
  if ('\\N' in time_str):
    return None
  if (not '.' in time_str):
    return None
  if (len(time_series) > 1):
    return int(ttime_seriess[0]) * 60 + float(time_series[1])
  else:
    return float(time_series[0])

In [ ]:
race_years = races.query('year  >= 2001')
race_ids = race_years['raceId']

for i in tqdm(race_ids):
  race_info = races.query(f'raceId == {i}')
  year = race_info['year'].item()
  if (year == 2021):
    continue
  circuit = race_info['circuitId'].item()
  if not os.path.exists(directory + f'races/{year}'):
      os.makedirs(directory + f'races/{year}')
  if os.path.exists(directory + f'races/{year}/race{i}.csv'):
    continue
  if (race_info['round'].item() > 1):
    d_standing_br = d_standings.query(f'raceId == {i-1}')
    c_standing_br = c_standings.query(f'raceId == {i-1}')
  else:
    previous_year = races.query(f'year == {year - 1}')
    previous_year = previous_year.sort_values(by=['round'])
    previous_year = previous_year.reset_index()
    previous_last_race = previous_year['raceId'].iloc[-1]
    d_standing_br  = d_standings.query(f'raceId == {previous_last_race}')
    c_standing_br  = c_standings.query(f'raceId == {previous_last_race}')
  quali_info = quali.query(f'raceId == {i}')
  race_laptimes = lap_times.query(f'raceId == {i}')
  race_pitstops = pit_stops.query(f'raceId == {i}')
  race_results = results.query(f'raceId == {i}')
  race_results_sorted = race_results.sort_values(by=['position'])
  race_results_sorted = race_results_sorted.reset_index()
  num_of_laps = race_results_sorted['laps'].iloc[0]
  race_results_sorted_grid = race_results.sort_values(by=['grid'])
  race_results_sorted_grid = race_results_sorted_grid.reset_index()
  # Make dataframe
  columns=['circuitId']
  for number in range(20):
    columns.extend([
            f'driverId{number+1}',
            f'driverStanding{number+1}',
            f'constructorStanding{number+1}',
            f'position{number+1}',
            f'inPit{number+1}',
            f'status{number+1}',
            f'laptime{number+1}'
    ])
  df = pd.DataFrame(columns=columns)
  # Each lap is a row, each race is a dataframe
  for lap_num in range(0, num_of_laps + 1):
    driver_ids = []
    d_s = []
    c_s = []
    pos = []
    pit = []
    statuses = []
    lps = []
    for id in race_results_sorted_grid['driverId']:
      driver_ids.append(id)
      constructorId = race_results.query(f'driverId == {id}')['constructorId'].item()
      driver_position = d_standing_br.query(f'driverId == {id}')['position']
      if (not driver_position.empty):
        d_s.append(driver_position.item())
      else:
        d_s.append(20)
      constructor_position = c_standing_br.query(f'constructorId == {constructorId}')['position']
      if (not constructor_position.empty):
        c_s.append(constructor_position.item())
      else:
        c_s.append(10)
      if (lap_num == 0):
        position = quali_info.query(f'driverId == {id}')['position']
        if (not position.empty):
          position = position.item()
        else:
          position = 21
      else:
        position = race_laptimes.query(f'driverId == {id} & lap == {lap_num}')['position']
        if (not position.empty): # position could be null
          position = position.item()
        else:
          position = 21 # 21 means retired
      pos.append(position)
      inp = race_pitstops.query(f'driverId == {id} & lap == {lap_num}')
      if (not inp.empty):
        pit.append(1)
      else:
        pit.append(0)

      if (lap_num == 0):
        q3_s = quali_info.query(f'driverId == {id}')['q3']
        if (not q3_s.empty):
          q3 = time_to_int(q3_s.item())
        else:
          q3 = None
        q2_s = quali_info.query(f'driverId == {id}')['q2']
        if (not q2_s.empty):
          q2 = time_to_int(q2_s.item())
        else:
          q2 = None
        q1_s = quali_info.query(f'driverId == {id}')['q1']
        if (not q1_s.empty):
          q1 = time_to_int(q1_s.item())
        else:
          q1 = None
        if (q3):
          lps.append(q3)
        elif (q2):
          lps.append(q2)
        elif (q1):
          lps.append(q1)
        else:
          lps.append(0)
        statuses.append(0) # 0 when in race or before race start
      elif (race_results.query(f'driverId == {id}')['laps'].item() <= lap_num): # check if driver has retired
        statuses.append(race_results.query(f'driverId == {id}')['statusId'].item())
        lps.append(0)
      else:
        statuses.append(0)
        time = race_laptimes.query(f'driverId == {id} & lap == {lap_num}')['time']
        if (not time.empty): 
          lps.append(time_to_int(time.item()))
        else: 
          lps.append(0)
    row = {}
    row['circuitId'] = circuit
    for j in range(len(driver_ids)):
        row[f'driverId{j+1}'] = driver_ids[j]
        row[f'driverStanding{j+1}'] = d_s[j]
        row[f'constructorStanding{j+1}'] = c_s[j]
        row[f'position{j+1}'] = pos[j]
        row[f'inPit{j+1}'] = pit[j]
        row[f'status{j+1}'] = statuses[j]
        row[f'laptime{j+1}'] = lps[j]
    df = df.append(row, ignore_index=True)
  df.to_csv(directory + f'races/{year}/race{i}.csv')

In [ ]:
years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for year in years:
  if not os.path.exists(directory + f'races_npy/{year}'):
        os.makedirs(directory + f'races_npy/{year}')

In [ ]:
years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for year in years:
  if not os.path.exists(directory + f'races_npy/{year}'):
      os.makedirs(directory + f'races_npy/{year}')

  current_year = os.listdir(directory + f'races/{year}/')
  counter = 1
  for round in current_year:
    
    current_race = pd.read_csv(directory + f'races/{year}/{round}')

    if (True):
      for i in range(len(current_race)-1):
        current = current_race.iloc[i].values[1:142]
        current[current != current] = 0
        cur_ret = np.zeros(130)
        cur_ret[int(current[0])] = 1
        lap_percentage = float(i) / (len(current_race)-1)
        _lp = np.zeros(1)
        _lp[0] = lap_percentage
        cur_ret = np.append(cur_ret, _lp)
        for k in range(1, 141, 7):
    
          d_id = driver_embed(driver_embed_id(int(current[k])))
          cur_ret = np.append(cur_ret, d_id)
    
          # 2 bits: top or top3 or top10
          d_s = np.zeros(3)
          _s = int(current[k+1])-1
          if (_s <= 1):
            d_s[0] = 1
          if (_s <=3):
            d_s[1] = 1
          if (_s <=10):
            d_s[2] = 1
          cur_ret = np.append(cur_ret, d_s)
    
          c_s = np.zeros(2)
          _c = int(current[k+2])-1
          if (_c <= 1):
            c_s[0] = 1
          if (_c <= 3):
            c_s[1] = 1
          cur_ret = np.append(cur_ret, c_s)
    
          pos = np.zeros(21)
          _p = int(current[k+3])-1
          if (_p >= 21):
            _p = 20
          pos[_p] = 1
          cur_ret = np.append(cur_ret, pos)
    
          pit = np.zeros(1)
          if (current[k+4] == 1):
            pit[0] = 1
          else :
            pit[0] = 0
          cur_ret = np.append(cur_ret, pit)
    
          stat = status_emb(int(current[k+5]))
          cur_ret = np.append(cur_ret, stat)
    
          laptime = laptime_embed(float(current[k+6]))
          cur_ret = np.append(cur_ret, laptime)
    
          rando = np.zeros(1)
          cur_ret = np.append(cur_ret, rando)
  
        if (i == 0):
          cur_race_in = cur_ret
        else:
          cur_race_in = np.vstack([cur_race_in, cur_ret])
      np.save(directory + f'races_npy/{year}/{counter}_in.npy', cur_race_in)
  
    if (True):
      for i in range(len(current_race)-1):
        next = current_race.iloc[i+1].values[1:142]
        next[next != next] = 0
        for k in range(1, 141, 7):
          pos = np.zeros(21)
          _p = int(next[k+3])-1
          if (_p >= 21):
            _p = 20
          pos[_p] = 1
          if (k == 1):
            next_ret = pos
          else:
            next_ret = np.append(next_ret, pos)
    
          pit = np.zeros(1)
          pit[0] = float(next[k+4]) / (len(current_race)-1)
          next_ret = np.append(next_ret, pit)
    
          stat = status_emb(int(next[k+5]))
          next_ret = np.append(next_ret, stat)
    
          laptime = status_emb(float(next[k+6]))
          next_ret = np.append(next_ret, laptime)

        if (i == 0):
          exp = next_ret
        else:
          exp = np.vstack([exp, next_ret])
      np.save(directory + f'races_npy/{year}/{counter}_exp.npy', exp)

    counter += 1
  print(year)

In [ ]:
def race_info(raceId):
  race_df = races.query(f'raceId  == {raceId}')
  if (race_df.empty):
    return None, None, None
  _year = race_df['year'].item()
  _round = race_df['round'].item()
  _circuitId = race_df['circuitId'].item()
  return _year, _round, _circuitId

def circuit_info(circuitId):
  circuit_df = circuits.query(f'circuitId  == {circuitId}')
  if (circuit_df.empty):
    return None, None, None
  _name = circuit_df['name'].item()
  _location = circuit_df['location'].item()
  _country = circuit_df['country'].item()
  return _name, _location, _country

def driver_info(id):
  drivers_df = drivers.query(f'driverId  == {id}')
  if (drivers_df.empty):
    return None, None, None, None, None, None
  _number = drivers_df['number'].item()
  _code = drivers_df['code'].item()
  _forename = drivers_df['forename'].item()
  _surname = drivers_df['surname'].item()
  _dob = drivers_df['dob'].item()
  _nationality = drivers_df['nationality'].item()
  return _number, _code, _forename, _surname, _dob, _nationality

def constructor_info(id):
  constructor_df = constructor.query(f'constructorId  == {id}')
  if (constructor_df.empty):
    return None, None
  _name = drivers['name'].item()
  _nationality = drivers['nationality'].item()
  return _name, _nationality

def status_info(id):
  status_df = status.query(f'statusId == {id}')
  if (status_df.empty):
    return None
  _sstr = status_df['status'].item()
  return _sstr